# Creating and Managing Vector Tables
__________________

This guide covers the basic methods for creating and managing tabular data. For full reference visit the [Vector API Documentation page](https://docs.descarteslabs.com/api/vector.html).

__Note__
Currently Vector is pre-installed on Workbench, however is not included with the main Descartes Labs Python client. To install Vector, run the following:

    !pip install descarteslabs-vector

In [ ]:
import descarteslabs as dl
from descarteslabs.vector import Feature, Table, models

In [ ]:
import geopandas as gpd
from pydantic import Field

## Creating a Vector Table

As with all other objects, tables must have an ID that is _unique to your organization_. For this example we will create a new table with our current user ID and share it with our organization:

In [ ]:
auth = dl.auth.Auth()
org = auth.payload["org"]
user_id = auth.namespace

Next we'll read in a sample vector file:

In [ ]:
gdf = gpd.read_parquet("data/countries.geoparquet")
gdf.plot(figsize=(10, 5))

And inspect its fields:

In [ ]:
gdf.head(5)

In [ ]:
gdf.info()

### Table Models

Vector allows the user to define a custom schema for each table. Below is the list of predefined base models which should be inherited to define the schema:

    models.VectorBaseModel # aspatial/tabular data only
    models.PointBaseModel # data containing point geometries
    models.MultiPointBaseModel # data containing multi-point geometries
    models.PolygonBaseModel # data containing polygon geometries
    models.MultiPolygonBaseModel # data containing multi-polygon geometries
    models.LineBaseModel # data containing line geometries
    models.MultiLineBaseModel # data containing multi-line geometries
    
_Important Notes on Table Models_:
* All models automatically generate a required uuid column
* If your data does not contain a geometry column (aspatial) the model should inherit from `models.VectorBaseModel` 
* If your data does contain a geometry column the appropriate model for the given geometry type must be selected, which will automatically generate a geometry column
* If a multi-geometry model is selected, all geometries will be promoted to multi-part

### Defining a Model's Fields
Once the appropriate base model has been determined, a custom schema can be created by inheriting the base model. Column names and data types can then be attributed to the custom model/schema through simple Python data typing. 

If the table has a geometry column, a spatial index will be created automatically. To specify the creation of an index on another column, use `pydantic.Field`, as in the below example:

In [ ]:
class CountryModel(models.MultiPolygonBaseModel):
    NAME: str = Field(json_schema_extra={"index": True})
    REGION_UN: str
    CONTINENT: str
    POP_EST: float
    LASTCENSUS: float

Next we create our table by passing our model in to the `Table.create()` method. We can also share access control lists (ACLs) at this time:

In [ ]:
# create the table with the default generic feature model
the_world = Table.create(
    f"the-world-{user_id}", name="The World", model=CountryModel, owners=[f"org:{org}"]
)

## Adding to and Modifying Tables
We can add a dataframe or geodataframe, depending on whether our data is spatial, by `Table.add()`. This will return a respective dataframe type with the new uuid field added:

In [ ]:
borders = the_world.add(gdf)
borders.head()

## Features

We can access and modify `Feature`s directly by their uuid through the equivalent `Feature.get()` or `Table.get_feature()` methods:

In [ ]:
feat1 = Feature.get(f"{the_world.id}:{borders.iloc[0].uuid}")
feat2 = the_world.get_feature(borders.iloc[0].uuid)
assert feat1.values == feat2.values

We can modify the feature and preserve the changes by `Feature.save()`:

In [ ]:
feat1.values["geometry"] = feat1.values["geometry"].convex_hull
feat1.values["POP_EST"] = 1000
feat1.save()

Retrieve the new feature to verify our changes:

In [ ]:
feat1 = Feature.get(f"{the_world.id}:{borders.iloc[0].uuid}")
feat1.values

## Managing Access to Tables

We can also modify the description and other ACLs, making sure we call `Table.save()`:

In [ ]:
the_world.description = "Country boundaries for the world."
the_world.readers.append("org:pga-tour")
the_world.save()
print(the_world.description)
print(the_world.readers)

## Deleting Tables
Finally, we can delete our table through `Table.delete()`:

In [ ]:
the_world.delete()